In [1]:
# My own Recurrent Neural Network(LSTM)

# Tensorflow
import tensorflow as tf
import numpy as np

#importing MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)




# Parameters ------------------------------------------------------------------------------

HLS = 256               # Hidden Layer Size
num_layers = 1          # Stacked Number of LSTM
learning_rate = 0.001
epochs = 10
batch_size = 256       # data / 1 batch
n_seq = 28

dropout = 0.5          # dropout
n_classes = 10

valid_data_size = 256


# ======================================================================================= #


def input_data(arr, n_seqs=1, n_steps=28):
    # in 1 image
    #small_batch_length = n_seqs * n_steps
    
    #for i in range(arr.shape[0]):
    #    arr[i,:,:,:] = arr[i,:small_batch_length,:,:]
    
    return 0 # randomly smapling method

def reshape_input(x, n_seq, n_data):
    X_in = tf.reshape(x, [-1,n_seq,n_data])
    return X_in


def fully_connected(input_layer, weights, biases, dropout):
    
    input_layer = tf.reshape(input_layer, [-1, weights['w1'].get_shape().as_list()[0]] ) # check
    
    
    hl1 = tf.add(tf.matmul(input_layer, weights['w1']), biases['b1'])
    hl1 = tf.nn.relu(hl1)
    hl1 = tf.nn.dropout(hl1, dropout)
    
    out = tf.add(tf.matmul(hl1, weights['wout']), biases['bout'])
    
    return out


def build_lstm(hidden_size, num_layers, batch_size, keep_prob):
    '''
    lstm_size : Size of Hidden Layers in the LSTM celber of LSTM Layers.ls
    num_layers : Number of LSTM Layers
    '''
    lstm = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
    #tf.contrib.BasicLSTMCell(hidden_size)
    
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob = keep_prob)
    
    
    # Stacking up
    cell = tf.contrib.rnn.MultiRNNCell( [drop]*num_layers )
    initial_state = cell.zero_state( batch_size, tf.float32 )
    
    return cell, initial_state

# ======================================================================================= #


# Weights & Biases ---------------------------------------------------------------------------

# -------------------- def Xavier_initializer -------------------- #
def Xavier_init(name, shape_in):
    return tf.get_variable(name, shape=shape_in, initializer=tf.contrib.layers.xavier_initializer())
# ---------------------------------------------------------------- #


weights = {
    'w1' : Xavier_init('w1',[n_seq*HLS, 1024]),
    'wout': Xavier_init('wout',[1024, n_classes])
}

biases = {

    'b1' : Xavier_init('b1',[1024]),
    'bout': Xavier_init('bout',[n_classes])
}




# input & output placeholder -------------------------------------------------------------------
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob_dropout = tf.placeholder(tf.float32)

# Model ----------------------------------------------------------------------------------------

rnn_cell, init_state = build_lstm(HLS, num_layers, batch_size, keep_prob_dropout) # works only with hyperparameter

X_in = reshape_input(x, n_seq=28, n_data=28)

output_data, state = tf.nn.dynamic_rnn(rnn_cell, X_in, initial_state=init_state, dtype=tf.float32) 
                                            # input x : batch_size, max_time, depth

logits = fully_connected(output_data, weights, biases, keep_prob_dropout)

# loss and optimizer ---------------------------------------------------------------------------

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy -------------------------------------------------------------------------------------

correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1)) # 1???? test later
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# initializer
init = tf.global_variables_initializer()

# Saver ----------------------------------------------------------------------------------------
save_file = './SaveDataRNN/test1.ckpt'
saver = tf.train.Saver()

# Training start -------------------------------------------------------------------------------
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            sess.run(optimizer, feed_dict={x:batch_x, y:batch_y, keep_prob_dropout: dropout}) # ...? drop out?
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob_dropout: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:valid_data_size],
                y: mnist.validation.labels[:valid_data_size],
                keep_prob_dropout: 1.})

            # print('Epoch = {:}, Batch = {:}, loss = {:.5f}, accuracy = {:.5f}'.format(epoch+1, batch+1, loss, valid_acc))
            # saver.save(sess, save_file)
        
        print('Epoch = {:}, Batch = {:}, loss = {:.5f}, accuracy = {:.5f}'.format(epoch+1, batch+1, loss, valid_acc))
        saver.save(sess, save_file)
    print('fuck')
    

/home/smnd/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Epoch = 1, Batch = 214, loss = 0.15921, accuracy = 0.96094
Epoch = 2, Batch = 214, loss = 0.04772, accuracy = 0.97266
Epoch = 3, Batch = 214, loss = 0.04263, accuracy = 0.97266
Epoch = 4, Batch = 214, loss = 0.04920, accuracy = 0.98047
Epoch = 5, Batch = 214, loss = 0.06713, accuracy = 0.97656
Epoch = 6, Batch = 214, loss = 0.04228, accuracy = 0.97656
Epoch = 7, Batch = 214, loss = 0.01150, accuracy = 0.97656
Epoch = 8, Batch = 214, loss = 0.00789, accuracy = 0.98438
Epoch = 9, Batch = 214, loss = 0.01954, accuracy = 0.98438
Epoch = 10, Batch = 214, loss = 0.01903, accuracy = 0.98438
fuck


In [6]:
import tensorflow as tf
import numpy as np

a = tf.constant(2, tf.float32)
b = tf.random_normal([3,2,2])
sess = tf.Session()


from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)
a = mnist.train.num_examples
print(mnist.train.num_examples)

np.shape(mnist.validation.images[:200])





Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
55000


(200, 28, 28, 1)